In [1]:
# The code was removed by Watson Studio for sharing.

## **Finding the Perfect Neighborhood in San Antonio, TX Informed by Foursquare Data**

### **D. Risius**  
### *1/5/2020*

### **Introduction:**
San Antonio, Texas is one of the fastest growing cities in the United States.  According to the [United States Census Bureau](https://www.census.gov/newsroom/press-releases/2018/estimates-cities.html), San Antonio topped the list of the fastest growing metro areas for 2017.  In previous analysis, we clustered and segmented neighborhoods in [Toronto](https://github.com/risiud/Coursera_Capstone/blob/master/Clustering%20Toronto%20Neighborhoods.ipynb) and New York city based on FourSquare venue data.  San Antonio is a very different city than either New York or Toronto.  For one, it is a very large city with relatively sparse population compared to the other cities.  According to [Wikopedia](https://en.wikipedia.org/wiki/San_Antonio), San Antonio city consists of around 1.5 million people within a land area of 461 square miles compared to 8.5 million for 303 square miles in [New York City](https://en.wikipedia.org/wiki/New_York_City) and 2.7 million for 243 square miles in [Toronto](https://en.wikipedia.org/wiki/Toronto).  The ethnicity of the three cities is also different.  San Antonio has a large hispanic influence with around 63% of residents of hispanic or latino origin.  New York is around 28% hispanic while Toronto is around 4% hispanic with a much larger proportion of Asian (40%) and European (48%) than San Antonio or New York.  Median housing prices between New York City and San Antonio are also very different.  According to [Zillow](https://www.zillow.com/research/data/), the median single-family home in December, 2019 was $477K in New York compared to around $204K in San Antonio.  If one can find the right neighborhood to live, San Antonio could provide a lot of value for the cost of living.  

### **Problem Statement:** 
Given a list of preferred criteria about a neighborhood, we would like to find an initial set of neighborhoods to begin searching for a new home in the San Antonio area.  Our initial set of criteria is as follows:

1. Median Home Price: I am looking for a single-family house within the $200K-$350K range. There are multiple neighborhoods both above and below this range so these will be eventually filtered out.  We would also like to find those neighborhoods where the median home price is increasing over time in case we would like to sell the home in the future
2. Good Schools: Since I have school-aged children, good schools in the neighborhood are very important.
3. Active lifestyle: Proximity to parks or other outdoor recreation is important.  The ability to walk or bike versus drive to these areas is also important.
4. Diversity of Activities: I would like the neighborhood to have a wide range of venues available nearby.  For instance, I wouldn't want all the top venues in the neighborhood to be gas stations or BBQ joints.  A wide range of venues such as dining, shopping, and recreation would be important.

Given our previous analysis clustering and segmenting neighborhoods using FourSquare data in New York City and Toronto, how does San Antonio, Texas compare in terms of most popular types of veneus?  If we wanted to move to a new neighborhood in San Antonio, can we use the FourSquare data for the different clusters to inform a decision on where to start our home search?

### **Data:** 
New York City and Toronto have well defined neighborhoods that helped us cluster the data.  San Antonio has some established neighborhoods, however many of the areas within the city are not defined within a particular neighborhood.  Therefore we can't use the same approach as we did with New York and Toronto as we would omit large portions of the city.  San Antonio consists of 87 seperate zip codes.  For analyzing San Antonio we will these zip codes intead and will map and cluster those using the geographical center of the zip code.  To get the geographic coordinates we used the website [San Antonio AreaConnect](https://sanantonio.areaconnect.com/zip2.htm?city=San%20Antonio&search=zip) which provides latitude/longitude coordinates for the various zip codes around San Antonio.  We will cluster these zip codes using the Foursquare location data similiar to the analysis in New York and Toronto.  Based on the cluster analysis, and our defined search criteria, we will recommend areas to start searching for homes in San Antonio.  The excerpt of the San Antonio zip code data below shows the data we will use for our analysis.  First we import all the necessary packages to read the data as a pandas dataframe and plot the geographic data on a map.

To analyze housing prices, we will use data from [Zillow Research](https://www.zillow.com/research/data/), which provides data on median home prices over time by zip code or other criteria. This data can assist of in narrowing down neighborhoods based on affordability and also show which growth over time.

For school information, there are multiple organizations that provide information and ratings on primary eduction.  For this project, we will use [TxSmartSchools.org](http://txsmartschools.org/).  TXSmartSchools uses academic, financial, and demographic data to identify school districts and campuses that produce high academic achievement while also maintaining cost-effective operations.  This data may assist us further narrowing our search based on proximity to quality schools.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [3]:
#geocode(query, country_codes=None, exactly_one=True, timeout=DEFAULT_SENTINEL)
#geocode('78251', country_codes='US', exactly_one=True, timeout=DEFAULT_SENTINEL)

address = '78251'

geolocator = Nominatim(user_agent="SA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Zip Code 78251 are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Zip Code 78251 are 29.4451859, -98.6817661.


Download the datasets

In [4]:
urlzips = "https://raw.githubusercontent.com/risiud/Coursera_Capstone/master/SanAntonioZips.csv"
urlhomes = "https://raw.githubusercontent.com/risiud/Coursera_Capstone/master/Zip_Zhvi_SingleFamilyResidence-SA.csv"
#urlschools = "https://raw.githubusercontent.com/risiud/Coursera_Capstone/master/Campus_Data_2017-18.csv"

neighborhoods = pd.read_csv(urlzips)
homes = pd.read_csv(urlhomes)
neighborhoods.head()

,Zipcode,City,State,AreaCode,County,Latitude,Longitude
0,78201,San Antonio,TX,210,Bexar,29.472,-98.537
1,78202,San Antonio,TX,210,Bexar,29.422,-98.466
2,78203,San Antonio,TX,210,Bexar,29.415,-98.462
3,78204,San Antonio,TX,210,Bexar,29.397,-98.500
4,78205,San Antonio,TX,210,Bexar,29.424,-98.487


In [5]:
homes.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,1998-08,1998-09,1998-10,1998-11,1998-12,1999-01,1999-02,1999-03,1999-04,1999-05,1999-06,1999-07,1999-08,1999-09,1999-10,1999-11,1999-12,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,2000-11,2000-12,2001-01,2001-02,2001-03,2001-04,2001-05,2001-06,2001-07,2001-08,2001-09,2001-10,2001-11,2001-12,2002-01,2002-02,2002-03,2002-04,2002-05,2002-06,2002-07,2002-08,2002-09,2002-10,2002-11,2002-12,2003-01,2003-02,2003-03,2003-04,2003-05,2003-06,2003-07,2003-08,2003-09,2003-10,2003-11,2003-12,2004-01,2004-02,2004-03,2004-04,2004-05,2004-06,2004-07,2004-08,2004-09,2004-10,2004-11,2004-12,2005-01,2005-02,2005-03,2005-04,2005-05,2005-06,2005-07,2005-08,2005-09,2005-10,2005-11,2005-12,2006-01,2006-02,2006-03,2006-04,2006-05,2006-06,2006-07,2006-08,2006-09,2006-10,2006-11,2006-12,2007-01,2007-02,2007-03,2007-04,2007-05,2007-06,2007-07,2007-08,2007-09,2007-10,2007-11,2007-12,2008-01,2008-02,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,2008-12,2009-01,2009-02,2009-03,2009-04,2009-05,2009-06,2009-07,2009-08,2009-09,2009-10,2009-11,2009-12,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,2010-10,2010-11,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
0,92271,78130,New Braunfels,TX,San Antonio-New Braunfels,Comal County,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,159235.0,160096.0,160098.0,161040.0,161441.0,162960.0,163312.0,163081.0,159660.0,156684.0,154940.0,157467.0,159861.0,159559.0,156548.0,152181.0,149399.0,147951.0,147897.0,148224.0,149282.0,150819.0,152407.0,152913.0,154095.0,155686.0,157363.0,158430.0,159290.0,159995.0,160432.0,161077.0,161908.0,162835.0,163430.0,163841.0,164009.0,164310.0,164733.0,165526.0,166185.0,166753.0,167077.0,167581.0,167949.0,168566.0,169207.0,169755.0,169802.0,170093.0,170838.0,172174.0,173266.0,174253.0,175212.0,176042.0,176621.0,176844.0,176332.0,175797.0,175215.0,175543.0,176056.0,176573.0,176974.0,177061.0,177256.0,177362.0,177773.0,177667.0,177381.0,176630.0,175864.0,175157.0,174784.0,174641.0,174858.0,174861.0,175173.0,175022.0,175199.0,175183.0,175470.0,175379.0,176244.0,176398.0,176657.0,175971.0,175266.0,174493.0,173522.0,172286.0,171741.0,171007.0,170868.0,169947.0,169217.0,168878.0,168875.0,169161.0,169189.0,169095.0,169014.0,169195.0,169335.0,169590.0,169918.0,170559.0,170752.0,170756.0,170950.0,171803.0,172588.0,173254.0,173773.0,174114.0,174726.0,175214.0,176200.0,176422.0,176537.0,176717.0,177264.0,178470.0,179407.0,180099.0,180379.0,180633.0,181144.0,181430.0,181540.0,181378.0,181544.0,182518.0,184109.0,185814.0,187474.0,188869.0,190340.

In [6]:
# The code was removed by Watson Studio for sharing.

,Unnamed: 0,Year,Campus Id,SchoolName,District Id,District Name,County Name,Region Number,Charter School,Alt Ed Type,Alt Ed Campus,Alternate Education,Disciplinary Alt Ed Program,Juvenile Justice Alt Ed,Grade Span,School Type,Composite Academic Progress Percentile (3 Year Avg),Composite Progress Z-Score (3 Year Avg),Composite Academic Progress Quintile (3 Year Avg),Math Progress Z-Score (3 Year Avg),Math Progress Z-Score,Math Progress Z-Score standard error,Reading Progress Z-Score (3 Year Avg),Reading Progress Z-Score,Reading Progress Z-Score standard error,TEA Accountability Rating,Smart Score,Spending Index,Spending Score,Cost-Adjusted Core Operating Expenditures Per Student (3 Year Avg),Cost-Adjusted Core Operating Expenditures Per Student,Core Operating Expenditures Per Student (not cost adjusted),Core Operating Expenditures (not cost adjusted),Core Payroll Expenditures (not cost adjusted),Campus Cost Deflator,Total Operating Expenditures Per Student (not cost adjusted),Urban County Indicator,ACS Comparable Wage Index,High School Comparable Wage Index,Enrollment,% Economically Disadvantaged,% LEP,% Special Education,% Student Mobility,% Students Early Childhood Education,% Students Pre-Kindergarten,% Students Kindergarten,% Students Grade 1,% Students Grade 2,% Students Grade 3,% Students Grade 4,% Students Grade 5,% Students Grade 6,% Students Grade 7,% Students Grade 8,% Students Grade 9,% Students Grade 10,% Students Grade 11,% Students Grade 12,% Students Bilingual/ESL,% Students Career and Technical,% Students Gifted and Talented,% Students Asian,% Students White,% Students African American,% Students Hispanic,% Students Pacific Islander,% Students American Indian,% Students Two Or More Races,% Students At-Risk,Fiscal Peers Group,Number of Campus Matches,Latitude,Longitude,Neighborhood
0,382,2018,15901001,ALAMO HEIGHTS H S,15901,ALAMO HEIGHTS ISD,BEXAR,20,N,,N,N,N,N,09-12,S,9.0,-0.129,1.0,-0.185063,-0.303317,0.110026,-0.073563,-0.081283,0.063798,Met Standard,2.0,Average Spending,3.0,6755.000000,6888.0,6814.0,10738985.0,10277652.0,1.01138,7960.0,1.0,0.917,0.972,1576,17.700001,2.3,7.4,6.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,27.700001,24.900000,24.000000,23.299999,2.3,37.099998,20.900000,3.6,52.200001,1.9,41.000000,0.1,0.6,0.6,25.900000,0.0,40.0,29.4913,-98.4646,78217
1,384,2018,15901041,ALAMO HEIGHTS J H,15901,ALAMO HEIGHTS ISD,BEXAR,20,N,,N,N,N,N,06-08,M,39.0,0.005,2.0,-0.004032,0.036091,0.105000,0.014587,0.034028,0.063195,Met Standard,2.0,High Spending,2.0,6387.000000,6815.0,6742.0,7415914.0,7105540.0,1.01138,7055.0,1.0,0.917,0.972,1100,20.500000,5.5,8.3,5.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.700001,34.200001,34.099998,0.000000,0.000000,0.000000,0.000000,7.6,0.000000,28.200001,2.5,52.000000,2.5,42.099998,0.2,0.3,0.4,28.500000,0.0,40.0,29.5011,-98.4601,78209
2,388,2018,15904001,HARLANDALE H S,15904,HARLANDALE ISD,BEXAR,20,N,,N,N,N,N,09-12,S,7.0,-0.148,1.0,-0.191388,-0.113796,0.063124,-0.104438,-0.090119,0.037749,Met Standard,2.0,Average Spending,3.0,7062.666667,7693.0,7614.0,14016927.0,12826592.0,1.01138,8778.0,1.0,0.917,0.972,1841,84.599998,14.0,11.8,16.299999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,30.799999,27.400000,23.400000,18.500000,13.6,85.000000,6.700000,0.1,1.200000,0.3,98.099998,0.0,0.1,0.2,71.000000,0.0,38.0,29.3654,-98.5021,78214
3,389,2018,15904002,MCCOLLUM H S,15904,HARLANDALE ISD,BEXAR,20,N,,N,N,N,N,09-12,S,4.0,-0.180,1.0,-0.250308,-0.147146,0.063613,-0.109703,-0.113306,0.038534,Met Standard,1.5,High Spending,2.0,7809.333333,8213.0,8128.0,12615109.0,11553218.0,1.01138,9319.0,1.0,0.917,0.972,1552,80.800003,7.0,12.3,16.900000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,29.200001,27.799999,21.299999,21.799999,6.8,87.199997,9.600000,0.1,3.300000,0.3,96.199997,0.0,0.0,0.1,65.099998,0.0,40.0,29.3435,-98.5123,78221
4,395,2018,15904011,HARLANDALE ISD STEM ECHS-ALAMO COL,15904,HARLANDALE ISD,BEXAR,20,N,,N,N,N,N,09-12,S,57.0,0.065,3.0,-0.057686,0

In [7]:
# Make the schools2 dataframe that takes the mean of the smart score by zip code
#data.groupby('month')['duration'].sum()
#schoolSum = schools.groupby('Neighborhood', as_index=False).agg({"Smart Score": "mean"})
schoolSum = schools.groupby('Neighborhood', as_index = False)[['Smart Score']].sum()
#schoolSum = schools.groupby('Neighborhood')['Smart Score'].mean()
print(len(schoolSum['Smart Score']))
schoolSum.head()

51


,Neighborhood,Smart Score
0,78023,2.5
1,78109,5.0
2,78148,7.5
3,78150,4.5
4,78154,3.0


In [8]:
#Lets categorize the zip codes by creating a new category on the median price in Dec 2019
#cutlist = pd.cut(homes['2019-12'], 6)
#cutlist.head
homes['price_bins'] = pd.cut(x=homes['2019-12'], bins=[0, 50000, 100000, 200000, 300000, 400000, 500000])
homes['price_labels']= pd.cut(x=homes['2019-12'], bins=[0, 100000, 200000, 300000, 400000, 500000], labels = [0,1,2,3,4] )

#Take the dataframe, keep only zip code, 2012-12 and 2019-12, pricebins and price-labels
homes = homes[['RegionName', '2012-12', '2019-12', 'price_bins', 'price_labels']]

homes.rename(columns = {'RegionName':'Neighborhood'}, inplace = True)
#homes.columns.values[0]='Neighborhood'

homes.head()

,Neighborhood,2012-12,2019-12,price_bins,price_labels
0,78130,174114.0,239955,"(200000, 300000]",2
1,78245,126281.0,186460,"(100000, 200000]",1
2,78240,138430.0,200246,"(200000, 300000]",2
3,78249,158196.0,224169,"(200000, 300000]",2
4,78254,163446.0,226363,"(200000, 300000]",2


In [9]:
#Now lets modify the schools
#drop missing lat/long values from schools list
schools = schools.dropna(subset=['Smart Score','Latitude', 'Longitude'])

schools.head()

#Take a subset with only the San Antonio Region to reduce the size
schools = schools[(schools['Region Number'] == 20) & (schools['Charter School'] == 'N') & ((schools['School Type'] == 'S') | (schools['School Type'] == 'M'))]
#schools = schools[(schools['Region Number'] == 20) & (schools['Charter School'] == 'N') & (schools['School Type'] == 'E')]

#Need to further limit by lat/long
schools = schools[(schools['Latitude'] <= 29.7420) & (schools['Latitude'] >= 29.303) & (schools['Longitude'] <= -98.2695) & (schools['Longitude'] >= -98.80)]
#schools = SA_merged[(SA_merged.price_labels >= 2) &  (SA_merged.price_labels <= 4)]

#Rename Campus Name Column
schools.rename(columns = {'Campus Name':'SchoolName'}, inplace = True)
schools.head()

,Unnamed: 0,Year,Campus Id,SchoolName,District Id,District Name,County Name,Region Number,Charter School,Alt Ed Type,Alt Ed Campus,Alternate Education,Disciplinary Alt Ed Program,Juvenile Justice Alt Ed,Grade Span,School Type,Composite Academic Progress Percentile (3 Year Avg),Composite Progress Z-Score (3 Year Avg),Composite Academic Progress Quintile (3 Year Avg),Math Progress Z-Score (3 Year Avg),Math Progress Z-Score,Math Progress Z-Score standard error,Reading Progress Z-Score (3 Year Avg),Reading Progress Z-Score,Reading Progress Z-Score standard error,TEA Accountability Rating,Smart Score,Spending Index,Spending Score,Cost-Adjusted Core Operating Expenditures Per Student (3 Year Avg),Cost-Adjusted Core Operating Expenditures Per Student,Core Operating Expenditures Per Student (not cost adjusted),Core Operating Expenditures (not cost adjusted),Core Payroll Expenditures (not cost adjusted),Campus Cost Deflator,Total Operating Expenditures Per Student (not cost adjusted),Urban County Indicator,ACS Comparable Wage Index,High School Comparable Wage Index,Enrollment,% Economically Disadvantaged,% LEP,% Special Education,% Student Mobility,% Students Early Childhood Education,% Students Pre-Kindergarten,% Students Kindergarten,% Students Grade 1,% Students Grade 2,% Students Grade 3,% Students Grade 4,% Students Grade 5,% Students Grade 6,% Students Grade 7,% Students Grade 8,% Students Grade 9,% Students Grade 10,% Students Grade 11,% Students Grade 12,% Students Bilingual/ESL,% Students Career and Technical,% Students Gifted and Talented,% Students Asian,% Students White,% Students African American,% Students Hispanic,% Students Pacific Islander,% Students American Indian,% Students Two Or More Races,% Students At-Risk,Fiscal Peers Group,Number of Campus Matches,Latitude,Longitude,Neighborhood
0,382,2018,15901001,ALAMO HEIGHTS H S,15901,ALAMO HEIGHTS ISD,BEXAR,20,N,,N,N,N,N,09-12,S,9.0,-0.129,1.0,-0.185063,-0.303317,0.110026,-0.073563,-0.081283,0.063798,Met Standard,2.0,Average Spending,3.0,6755.000000,6888.0,6814.0,10738985.0,10277652.0,1.01138,7960.0,1.0,0.917,0.972,1576,17.700001,2.3,7.4,6.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,27.700001,24.900000,24.000000,23.299999,2.3,37.099998,20.900000,3.6,52.200001,1.9,41.000000,0.1,0.6,0.6,25.900000,0.0,40.0,29.4913,-98.4646,78217
1,384,2018,15901041,ALAMO HEIGHTS J H,15901,ALAMO HEIGHTS ISD,BEXAR,20,N,,N,N,N,N,06-08,M,39.0,0.005,2.0,-0.004032,0.036091,0.105000,0.014587,0.034028,0.063195,Met Standard,2.0,High Spending,2.0,6387.000000,6815.0,6742.0,7415914.0,7105540.0,1.01138,7055.0,1.0,0.917,0.972,1100,20.500000,5.5,8.3,5.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.700001,34.200001,34.099998,0.000000,0.000000,0.000000,0.000000,7.6,0.000000,28.200001,2.5,52.000000,2.5,42.099998,0.2,0.3,0.4,28.500000,0.0,40.0,29.5011,-98.4601,78209
2,388,2018,15904001,HARLANDALE H S,15904,HARLANDALE ISD,BEXAR,20,N,,N,N,N,N,09-12,S,7.0,-0.148,1.0,-0.191388,-0.113796,0.063124,-0.104438,-0.090119,0.037749,Met Standard,2.0,Average Spending,3.0,7062.666667,7693.0,7614.0,14016927.0,12826592.0,1.01138,8778.0,1.0,0.917,0.972,1841,84.599998,14.0,11.8,16.299999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,30.799999,27.400000,23.400000,18.500000,13.6,85.000000,6.700000,0.1,1.200000,0.3,98.099998,0.0,0.1,0.2,71.000000,0.0,38.0,29.3654,-98.5021,78214
3,389,2018,15904002,MCCOLLUM H S,15904,HARLANDALE ISD,BEXAR,20,N,,N,N,N,N,09-12,S,4.0,-0.180,1.0,-0.250308,-0.147146,0.063613,-0.109703,-0.113306,0.038534,Met Standard,1.5,High Spending,2.0,7809.333333,8213.0,8128.0,12615109.0,11553218.0,1.01138,9319.0,1.0,0.917,0.972,1552,80.800003,7.0,12.3,16.900000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,29.200001,27.799999,21.299999,21.799999,6.8,87.199997,9.600000,0.1,3.300000,0.3,96.199997,0.0,0.0,0.1,65.099998,0.0,40.0,29.3435,-98.5123,78221
4,395,2018,15904011,HARLANDALE ISD STEM ECHS-ALAMO COL,15904,HARLANDALE ISD,BEXAR,20,N,,N,N,N,N,09-12,S,57.0,0.065,3.0,-0.057686,0

In [10]:
#print(schools.Latitude, schools.Longitude)
#pd.isna(schools['Smart Score'])
print(len(schools.Latitude))
schools.Longitude.dtype

110


dtype('float64')

In [11]:
#Now lets figure out how to get the postal code on this dataset
import re
geolocator2 = Nominatim(user_agent="Get_Zip_Codes")

#That works, lets try to do a for loop
    
postcode = pd.DataFrame(columns = ('SchoolName', 'Latitude','Longitude', 'Zipcode'))
code = []

for index, row in schools.iterrows():
    lat = str(schools.Latitude[index])
    lon = str(schools.Longitude[index])
    location = geolocator2.reverse("{} , {}".format(lat,lon))
    address = location.address
    zipcode = re.match('^.*(?P<zipcode>\d{5}).*$', address).groupdict()['zipcode']
    print(index, zipcode)
    code.append(zipcode)

#>>> shepherd = "Mary"
#>>> age = 32
#>>> stuff_in_string = "Shepherd {} is {} years old.".format(shepherd, age)
# Creating the first Dataframe using dictionary 
schools['Neighborhood'] = code

0 78217
1 78209
2 78214
3 78221
4 78214
5 78214
6 78211
7 78221
8 78221
9 78226
10 78228
11 78226
12 78237
13 78228
14 78150
15 78150
16 78224
17 78204
18 78212
19 78205
20 78210
21 78220
22 78201
23 78207
24 78212
25 78220
26 78204
27 78207
28 78228
29 78226
30 78210
31 78203
32 78207
33 78223
34 78212
35 78201
36 78207
37 78202
38 78224
39 78211
40 78242
41 78224
42 78213
43 78217
44 78216
45 78218
46 78247
47 78258
48 78213
49 78259
50 78213
51 78209
52 78218
53 78230
54 78230
55 78218
56 78233
57 78232
58 78247
59 78258
60 78259
61 78258
62 78247
63 78259
64 78263
65 78263
66 78222
67 78252
68 78242
69 78248
70 78238
71 78227
72 78240
73 78249
74 78229
75 78253
76 78023
77 78251
78 78251
79 78249
80 78253
81 78238
82 78227
83 78228
84 78227
85 78230
86 78245
87 78250
88 78240
89 78250
90 78249
91 78251
92 78250
93 78249
94 78253
95 78254
96 78251
97 78256
98 78253
99 78254
100 78238
101 78109
102 78244
103 78148
104 78219
105 78148
106 78244
107 78244
108 78109
109 78154


In [12]:
#Lets save the dataframe
# Import the lib
#from project_lib import Project
#project = Project(sc,"<ProjectId>", "<ProjectToken>")

# let's assume you have the pandas DataFrame  pandas_df which contains the data
# you want to save in your object storage as a csv file
#project.save_data("schools.csv", schools.to_csv())

# When we get it, save the overall neighborhood file so we don't need to constantly run

# the function returns a dict which contains the asset_id, bucket_name and file_name
# upon successful saving of the data

#Now need to summarize the dataframe on mean of smart score by zip code


schools.head()

,Unnamed: 0,Year,Campus Id,SchoolName,District Id,District Name,County Name,Region Number,Charter School,Alt Ed Type,Alt Ed Campus,Alternate Education,Disciplinary Alt Ed Program,Juvenile Justice Alt Ed,Grade Span,School Type,Composite Academic Progress Percentile (3 Year Avg),Composite Progress Z-Score (3 Year Avg),Composite Academic Progress Quintile (3 Year Avg),Math Progress Z-Score (3 Year Avg),Math Progress Z-Score,Math Progress Z-Score standard error,Reading Progress Z-Score (3 Year Avg),Reading Progress Z-Score,Reading Progress Z-Score standard error,TEA Accountability Rating,Smart Score,Spending Index,Spending Score,Cost-Adjusted Core Operating Expenditures Per Student (3 Year Avg),Cost-Adjusted Core Operating Expenditures Per Student,Core Operating Expenditures Per Student (not cost adjusted),Core Operating Expenditures (not cost adjusted),Core Payroll Expenditures (not cost adjusted),Campus Cost Deflator,Total Operating Expenditures Per Student (not cost adjusted),Urban County Indicator,ACS Comparable Wage Index,High School Comparable Wage Index,Enrollment,% Economically Disadvantaged,% LEP,% Special Education,% Student Mobility,% Students Early Childhood Education,% Students Pre-Kindergarten,% Students Kindergarten,% Students Grade 1,% Students Grade 2,% Students Grade 3,% Students Grade 4,% Students Grade 5,% Students Grade 6,% Students Grade 7,% Students Grade 8,% Students Grade 9,% Students Grade 10,% Students Grade 11,% Students Grade 12,% Students Bilingual/ESL,% Students Career and Technical,% Students Gifted and Talented,% Students Asian,% Students White,% Students African American,% Students Hispanic,% Students Pacific Islander,% Students American Indian,% Students Two Or More Races,% Students At-Risk,Fiscal Peers Group,Number of Campus Matches,Latitude,Longitude,Neighborhood
0,382,2018,15901001,ALAMO HEIGHTS H S,15901,ALAMO HEIGHTS ISD,BEXAR,20,N,,N,N,N,N,09-12,S,9.0,-0.129,1.0,-0.185063,-0.303317,0.110026,-0.073563,-0.081283,0.063798,Met Standard,2.0,Average Spending,3.0,6755.000000,6888.0,6814.0,10738985.0,10277652.0,1.01138,7960.0,1.0,0.917,0.972,1576,17.700001,2.3,7.4,6.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,27.700001,24.900000,24.000000,23.299999,2.3,37.099998,20.900000,3.6,52.200001,1.9,41.000000,0.1,0.6,0.6,25.900000,0.0,40.0,29.4913,-98.4646,78217
1,384,2018,15901041,ALAMO HEIGHTS J H,15901,ALAMO HEIGHTS ISD,BEXAR,20,N,,N,N,N,N,06-08,M,39.0,0.005,2.0,-0.004032,0.036091,0.105000,0.014587,0.034028,0.063195,Met Standard,2.0,High Spending,2.0,6387.000000,6815.0,6742.0,7415914.0,7105540.0,1.01138,7055.0,1.0,0.917,0.972,1100,20.500000,5.5,8.3,5.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.700001,34.200001,34.099998,0.000000,0.000000,0.000000,0.000000,7.6,0.000000,28.200001,2.5,52.000000,2.5,42.099998,0.2,0.3,0.4,28.500000,0.0,40.0,29.5011,-98.4601,78209
2,388,2018,15904001,HARLANDALE H S,15904,HARLANDALE ISD,BEXAR,20,N,,N,N,N,N,09-12,S,7.0,-0.148,1.0,-0.191388,-0.113796,0.063124,-0.104438,-0.090119,0.037749,Met Standard,2.0,Average Spending,3.0,7062.666667,7693.0,7614.0,14016927.0,12826592.0,1.01138,8778.0,1.0,0.917,0.972,1841,84.599998,14.0,11.8,16.299999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,30.799999,27.400000,23.400000,18.500000,13.6,85.000000,6.700000,0.1,1.200000,0.3,98.099998,0.0,0.1,0.2,71.000000,0.0,38.0,29.3654,-98.5021,78214
3,389,2018,15904002,MCCOLLUM H S,15904,HARLANDALE ISD,BEXAR,20,N,,N,N,N,N,09-12,S,4.0,-0.180,1.0,-0.250308,-0.147146,0.063613,-0.109703,-0.113306,0.038534,Met Standard,1.5,High Spending,2.0,7809.333333,8213.0,8128.0,12615109.0,11553218.0,1.01138,9319.0,1.0,0.917,0.972,1552,80.800003,7.0,12.3,16.900000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,29.200001,27.799999,21.299999,21.799999,6.8,87.199997,9.600000,0.1,3.300000,0.3,96.199997,0.0,0.0,0.1,65.099998,0.0,40.0,29.3435,-98.5123,78221
4,395,2018,15904011,HARLANDALE ISD STEM ECHS-ALAMO COL,15904,HARLANDALE ISD,BEXAR,20,N,,N,N,N,N,09-12,S,57.0,0.065,3.0,-0.057686,0

In [13]:
#Join the homes and neighborhoods files
neighborhoods = neighborhoods.join(homes.set_index('Neighborhood'), on='Zipcode')

neighborhoods.head()

,Zipcode,City,State,AreaCode,County,Latitude,Longitude,2012-12,2019-12,price_bins,price_labels
0,78201,San Antonio,TX,210,Bexar,29.472,-98.537,88509.0,156320.0,"(100000, 200000]",1
1,78202,San Antonio,TX,210,Bexar,29.422,-98.466,60016.0,129942.0,"(100000, 200000]",1
2,78203,San Antonio,TX,210,Bexar,29.415,-98.462,71213.0,150560.0,"(100000, 200000]",1
3,78204,San Antonio,TX,210,Bexar,29.397,-98.500,77524.0,137329.0,"(100000, 200000]",1
4,78205,San Antonio,TX,210,Bexar,29.424,-98.487,184158.0,259457.0,"(200000, 300000]",2


In [14]:
# Now join the neighborhood and school files
#Join the schools and neighborhoods files
neighborhoods = neighborhoods.join(schoolSum.set_index('Neighborhood'), on = 'Zipcode')
neighborhoods.head()

,Zipcode,City,State,AreaCode,County,Latitude,Longitude,2012-12,2019-12,price_bins,price_labels,Smart Score
0,78201,San Antonio,TX,210,Bexar,29.472,-98.537,88509.0,156320.0,"(100000, 200000]",1,4.0
1,78202,San Antonio,TX,210,Bexar,29.422,-98.466,60016.0,129942.0,"(100000, 200000]",1,3.0
2,78203,San Antonio,TX,210,Bexar,29.415,-98.462,71213.0,150560.0,"(100000, 200000]",1,1.5
3,78204,San Antonio,TX,210,Bexar,29.397,-98.500,77524.0,137329.0,"(100000, 200000]",1,3.0
4,78205,San Antonio,TX,210,Bexar,29.424,-98.487,184158.0,259457.0,"(200000, 300000]",2,2.0


In [15]:
print('The dataframe has {} Zip Codes.'.format(
        len(neighborhoods['Zipcode'].unique())
    )
)

The dataframe has 87 Zip Codes.


In [16]:
address = 'San Antonio, TX'

geolocator = Nominatim(user_agent="SA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Antonio are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Antonio are 29.4246002, -98.4951405.


In [17]:
# create map of San Antonio using latitude and longitude values
map_SA = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, zipcode in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Zipcode']):
    label = '{}'.format(zipcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_SA)  
    
map_SA

In [18]:
# Create a map with school data also
# Issues with missing Lat/Long data
# create map of San Antonio Schools using latitude and longitude values
#map_SA_school = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
#for lat, lng, school in zip(schools['Latitude'], schools['Longitude'], schools['Campus Name']):
#    label = '{}'.format(school)
#    label = folium.Popup(label, parse_html=True)
#    folium.CircleMarker(
#        [lat, lng],
#        radius=5,
#        popup=label,
#        color='blue',
#        fill=True,
#        fill_color='#3186cc',
#        fill_opacity=0.7,
#        parse_html=False).add_to(map_SA_school)  
    
#map_SA_school

In [19]:
# Define Four-Square Credentials

CLIENT_ID = '2HVAO4WMSHFLNNF1LGIQJBZYIVPSF3WBJPPDORAVO1M5NSHN' # your Foursquare ID
CLIENT_SECRET = 'YXKMVUBPMFSNUKICEK0QAEIT0R0HK3GXS1FP5CY2NAQ12SNT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2HVAO4WMSHFLNNF1LGIQJBZYIVPSF3WBJPPDORAVO1M5NSHN
CLIENT_SECRET:YXKMVUBPMFSNUKICEK0QAEIT0R0HK3GXS1FP5CY2NAQ12SNT


In [20]:
# Lets explore zip code 78251
neighborhoods.loc[46, 'Zipcode']


78247

In [21]:
# Get the latitude and longitude values of zip 78251

neighborhood_latitude = neighborhoods.loc[46, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[46, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[46, 'Zipcode'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of 78247 are 29.588, -98.402.


In [22]:
# Now lets get the top 100 venues in zip 78251 within a 2500m radius

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 2500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=2HVAO4WMSHFLNNF1LGIQJBZYIVPSF3WBJPPDORAVO1M5NSHN&client_secret=YXKMVUBPMFSNUKICEK0QAEIT0R0HK3GXS1FP5CY2NAQ12SNT&v=20180605&ll=29.588,-98.402&radius=2500&limit=100'

In [23]:
# Send the get requests and examine the results

results = requests.get(url).json()
#results

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
# Now clean the data and put it in a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

#nearby_venues

In [26]:
#Lets get the top 100 venues in each neighborhood

def getNearbyVenues(names, latitudes, longitudes, radius=2500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
SA_venues = getNearbyVenues(names=neighborhoods['Zipcode'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

78201
78202
78203
78204
78205
78206
78207
78208
78209
78210
78211
78212
78213
78214
78215
78216
78217
78218
78219
78220
78221
78222
78223
78224
78225
78226
78227
78228
78229
78230
78231
78232
78233
78234
78235
78236
78237
78238
78239
78240
78241
78242
78243
78244
78245
78246
78247
78248
78249
78250
78251
78252
78253
78254
78255
78256
78257
78258
78259
78260
78261
78262
78263
78264
78265
78266
78268
78269
78270
78275
78278
78280
78283
78284
78285
78286
78287
78288
78289
78291
78292
78293
78294
78295
78296
78297
78298


In [28]:
# Check the shape of the data frame
print(SA_venues.shape)
SA_venues.head()

(6015, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,78201,29.472,-98.537,Original Donut Shop,29.472703,-98.534598,Donut Shop
1,78201,29.472,-98.537,Restaurant Depot,29.473163,-98.535505,Kitchen Supply Store
2,78201,29.472,-98.537,Pancake Joes,29.464605,-98.543695,Breakfast Spot
3,78201,29.472,-98.537,Taqueria Puro Jalisco,29.479385,-98.541358,Mexican Restaurant
4,78201,29.472,-98.537,Jacala Mexican Restaurant,29.468267,-98.525847,Mexican Restaurant


In [29]:
# See how many venues were returned by neighborhood
SA_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
78201,100,100,100,100,100,100
78202,100,100,100,100,100,100
78203,72,72,72,72,72,72
78204,100,100,100,100,100,100
78205,100,100,100,100,100,100
78206,100,100,100,100,100,100
78207,75,75,75,75,75,75
78208,100,100,100,100,100,100
78209,100,100,100,100,100,100


In [30]:
# Find out how many unique categories there are from the returned venues
print('There are {} uniques categories.'.format(len(SA_venues['Venue Category'].unique())))

There are 299 uniques categories.


In [31]:
# Analyze each of the zip codes

# one hot encoding
SA_onehot = pd.get_dummies(SA_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
SA_onehot['Neighborhood'] = SA_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [SA_onehot.columns[-1]] + list(SA_onehot.columns[:-1])
SA_onehot = SA_onehot[fixed_columns]

SA_onehot.head()

,Zoo Exhibit,Accessories Store,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Canal,Candy Store,Caribbean Restaurant,Casino,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,College Football Field,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Credit Union,Cupcake Shop,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lawyer,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Non-Profit,Noodle House,Office,Optical Shop,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Pool,Pool Hall,Print Shop,Private School,Pub,Public Art,RV Park,Real Estate Office,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Storage Facility,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tanning Salon,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tiki Bar

In [32]:
# Examine the dataframe size
SA_onehot.shape

(6015, 299)

In [33]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
SA_grouped = SA_onehot.groupby('Neighborhood').mean().reset_index()
SA_grouped

,Neighborhood,Zoo Exhibit,Accessories Store,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Canal,Candy Store,Caribbean Restaurant,Casino,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,College Football Field,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Credit Union,Cupcake Shop,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lawyer,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Nightlife Spot,Non-Profit,Noodle House,Office,Optical Shop,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Pool,Pool Hall,Print Shop,Private School,Pub,Public Art,RV Park,Real Estate Office,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Storage Facility,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tanning Salon,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tiki Bar

In [34]:
# Confirm the new size
SA_grouped.shape

(87, 299)

In [35]:
# Print each zip code with the top 5 venues

num_top_venues = 5

for hood in SA_grouped['Neighborhood']:
    print("----", hood, "----")
    temp = SA_grouped[SA_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- 78201 ----
                  venue  freq
0    Mexican Restaurant  0.14
1        Discount Store  0.08
2  Fast Food Restaurant  0.06
3     Convenience Store  0.05
4           Pizza Place  0.05


---- 78202 ----
            venue  freq
0           Hotel  0.13
1      Steakhouse  0.04
2         Theater  0.04
3  Sandwich Place  0.04
4       BBQ Joint  0.04


---- 78203 ----
                venue  freq
0               Hotel  0.12
1  Mexican Restaurant  0.06
2         Coffee Shop  0.04
3        Burger Joint  0.04
4           BBQ Joint  0.04


---- 78204 ----
                venue  freq
0  Mexican Restaurant  0.13
1         Gas Station  0.05
2  Seafood Restaurant  0.04
3           BBQ Joint  0.04
4             Brewery  0.03


---- 78205 ----
        venue  freq
0       Hotel  0.21
1  Steakhouse  0.05
2         Bar  0.05
3     Theater  0.04
4       Plaza  0.03


---- 78206 ----
                venue  freq
0               Hotel  0.15
1  Mexican Restaurant  0.05
2               Plaza  0.04
3 

In [36]:
# Now we put the venues in a pandas dataframe

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [50]:
# Create a new dataframe with the top 10 venues for each zip code

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = SA_grouped['Neighborhood']

for ind in np.arange(SA_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(SA_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,78201,Mexican Restaurant,Discount Store,Fast Food Restaurant,Pizza Place,Convenience Store,Ice Cream Shop,Sandwich Place,Burger Joint,Coffee Shop,Gym / Fitness Center
1,78202,Hotel,Steakhouse,BBQ Joint,Sandwich Place,Theater,Cocktail Bar,Ice Cream Shop,Museum,Coffee Shop,Beer Garden
2,78203,Hotel,Mexican Restaurant,Burger Joint,Coffee Shop,BBQ Joint,History Museum,Hotel Bar,Steakhouse,Brewery,Sports Bar
3,78204,Mexican Restaurant,Gas Station,BBQ Joint,Seafood Restaurant,Sandwich Place,Brewery,Discount Store,Park,Beer Garden,Bar
4,78205,Hotel,Steakhouse,Bar,Theater,Sandwich Place,Plaza,Dessert Shop,Restaurant,Mexican Restaurant,Concert Hall


In [51]:
# Now let's cluster the neighborhoods

# set number of clusters
kclusters = 6

SA_grouped_clustering = SA_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(SA_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 2, 1, 2, 2, 1, 0, 0, 1], dtype=int32)

In [52]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [53]:
#Merge the dataframes
SA_merged = neighborhoods

# merge SA_grouped with SA_data to add latitude/longitude for each neighborhood
SA_merged = SA_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Zipcode')

SA_merged.head() # check the last columns!

,Zipcode,City,State,AreaCode,County,Latitude,Longitude,2012-12,2019-12,price_bins,price_labels,Smart Score,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,78201,San Antonio,TX,210,Bexar,29.472,-98.537,88509.0,156320.0,"(100000, 200000]",1,4.0,1,Mexican Restaurant,Discount Store,Fast Food Restaurant,Pizza Place,Convenience Store,Ice Cream Shop,Sandwich Place,Burger Joint,Coffee Shop,Gym / Fitness Center
1,78202,San Antonio,TX,210,Bexar,29.422,-98.466,60016.0,129942.0,"(100000, 200000]",1,3.0,2,Hotel,Steakhouse,BBQ Joint,Sandwich Place,Theater,Cocktail Bar,Ice Cream Shop,Museum,Coffee Shop,Beer Garden
2,78203,San Antonio,TX,210,Bexar,29.415,-98.462,71213.0,150560.0,"(100000, 200000]",1,1.5,2,Hotel,Mexican Restaurant,Burger Joint,Coffee Shop,BBQ Joint,History Museum,Hotel Bar,Steakhouse,Brewery,Sports Bar
3,78204,San Antonio,TX,210,Bexar,29.397,-98.500,77524.0,137329.0,"(100000, 200000]",1,3.0,1,Mexican Restaurant,Gas Station,BBQ Joint,Seafood Restaurant,Sandwich Place,Brewery,Discount Store,Park,Beer Garden,Bar
4,78205,San Antonio,TX,210,Bexar,29.424,-98.487,184158.0,259457.0,"(200000, 300000]",2,2.0,2,Hotel,Steakhouse,Bar,Theater,Sandwich Place,Plaza,Dessert Shop,Restaurant,Mexican Restaurant,Concert Hall


In [55]:
# create map of the clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(SA_merged['Latitude'], SA_merged['Longitude'], SA_merged['Zipcode'], SA_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1.0).add_to(map_clusters)
       
map_clusters

In [86]:
# Now lets filter only those in price labels 2-3 (200K-400k)
SA_merged_filter = SA_merged[(SA_merged['2019-12'] >= 185000) &  (SA_merged['2019-12'] <= 350000) ]
#surveys_df[(surveys_df.year >= 1980) & (surveys_df.year <= 1985)]
SA_merged_filter.head()
#len(SA_merged_filter.Zipcode)

,Zipcode,City,State,AreaCode,County,Latitude,Longitude,2012-12,2019-12,price_bins,price_labels,Smart Score,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,78205,San Antonio,TX,210,Bexar,29.424,-98.487,184158.0,259457.0,"(200000, 300000]",2,2.0,2,Hotel,Steakhouse,Bar,Theater,Sandwich Place,Plaza,Dessert Shop,Restaurant,Mexican Restaurant,Concert Hall
11,78212,San Antonio,TX,210,Bexar,29.462,-98.494,109080.0,189900.0,"(100000, 200000]",1,8.5,1,Mexican Restaurant,Burger Joint,Bar,Park,Gay Bar,American Restaurant,Pizza Place,Coffee Shop,Taco Place,Cajun / Creole Restaurant
14,78215,San Antonio,TX,210,Bexar,29.438,-98.481,123270.0,227805.0,"(200000, 300000]",2,NaN,2,Hotel,Bar,Burger Joint,Restaurant,Cocktail Bar,New American Restaurant,Bakery,Coffee Shop,Sandwich Place,Mexican Restaurant
15,78216,San Antonio,TX,210,Bexar,29.534,-98.489,156180.0,233606.0,"(200000, 300000]",2,2.0,0,Hotel,Mexican Restaurant,Department Store,American Restaurant,Clothing Store,Cosmetics Shop,Seafood Restaurant,Toy / Game Store,Sporting Goods Shop,Fast Food Restaurant
28,78229,San Antonio,TX,210,Bexar,29.501,-98.574,134635.0,194267.0,"(100000, 200000]",1,3.5,0,Mexican Restaurant,Coffee Shop,Sandwich Place,Chinese Restaurant,Bar,Pharmacy,Bakery,Indian Restaurant,Pizza Place,Fried Chicken Joint


Lets examine the clusters

In [87]:
# create map of the clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, price, score in zip(SA_merged_filter['Latitude'], SA_merged_filter['Longitude'], SA_merged_filter['Zipcode'], 
                                                SA_merged_filter['Cluster Labels'], SA_merged_filter['price_labels'], SA_merged['Smart Score']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) + ' Price $' + str(price) + ' Smart Score ' + str(score), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters

In [91]:
#Take the dataframe, keep only zip code, 2012-12 and 2019-12, pricebins and price-labels
finalFilter = SA_merged_filter.drop(['City', 'State', 'AreaCode', 'County', 'Latitude', 'Longitude'], axis = 1)
finalFilter # check the last columns!
#SA_merged_filter

,Zipcode,2012-12,2019-12,price_bins,price_labels,Smart Score,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,78205,184158.0,259457.0,"(200000, 300000]",2,2.0,2,Hotel,Steakhouse,Bar,Theater,Sandwich Place,Plaza,Dessert Shop,Restaurant,Mexican Restaurant,Concert Hall
11,78212,109080.0,189900.0,"(100000, 200000]",1,8.5,1,Mexican Restaurant,Burger Joint,Bar,Park,Gay Bar,American Restaurant,Pizza Place,Coffee Shop,Taco Place,Cajun / Creole Restaurant
14,78215,123270.0,227805.0,"(200000, 300000]",2,NaN,2,Hotel,Bar,Burger Joint,Restaurant,Cocktail Bar,New American Restaurant,Bakery,Coffee Shop,Sandwich Place,Mexican Restaurant
15,78216,156180.0,233606.0,"(200000, 300000]",2,2.0,0,Hotel,Mexican Restaurant,Department Store,American Restaurant,Clothing Store,Cosmetics Shop,Seafood Restaurant,Toy / Game Store,Sporting Goods Shop,Fast Food Restaurant
28,78229,134635.0,194267.0,"(100000, 200000]",1,3.5,0,Mexican Restaurant,Coffee Shop,Sandwich Place,Chinese Restaurant,Bar,Pharmacy,Bakery,Indian Restaurant,Pizza Place,Fried Chicken Joint
29,78230,198422.0,278380.0,"(200000, 300000]",2,6.0,0,Mexican Restaurant,Coffee Shop,Burger Joint,Sandwich Place,Chinese Restaurant,Sushi Restaurant,Bar,Grocery Store,Fast Food Restaurant,Gym
30,78231,234642.0,326961.0,"(300000, 400000]",3,NaN,0,Pizza Place,Pharmacy,Gym / Fitness Center,Gas Station,Video Store,Coffee Shop,Convenience Store,Spa,Cosmetics Shop,Fast Food Restaurant
31,78232,199577.0,280971.0,"(200000, 300000]",2,3.0,0,Mexican Restaurant,Convenience Store,Burger Joint,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Ice Cream Shop,Taco Place,Chinese Restaurant,Pizza Place
39,78240,138430.0,200246.0,"(200000, 300000]",2,4.5,0,Mexican Restaurant,Video Store,Pizza Place,Chinese Restaurant,Sandwich Place,Discount Store,Salon / Barbershop,Pharmacy,Café,Park
44,78245,126281.0,186460.0,"(100000, 200000]",1,2.5,4,Video Store,Pool,Golf Course,Pharmacy,Food & Drink Shop,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant


In [97]:
# Now lets filter only the final five
finalFilter = SA_merged_filter[(SA_merged_filter['Zipcode'] == 78254) |  (SA_merged_filter['Zipcode'] == 78253) | (SA_merged_filter['Zipcode'] == 78231) |  (SA_merged_filter['Zipcode'] == 78249) |  (SA_merged_filter['Zipcode'] == 78212) |  (SA_merged_filter['Zipcode'] == 78251)]
#surveys_df[(surveys_df.year >= 1980) & (surveys_df.year <= 1985)]
finalFilter
#len(SA_merged_filter.Zipcode)

,Zipcode,City,State,AreaCode,County,Latitude,Longitude,2012-12,2019-12,price_bins,price_labels,Smart Score,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,78212,San Antonio,TX,210,Bexar,29.462,-98.494,109080.0,189900.0,"(100000, 200000]",1,8.5,1,Mexican Restaurant,Burger Joint,Bar,Park,Gay Bar,American Restaurant,Pizza Place,Coffee Shop,Taco Place,Cajun / Creole Restaurant
30,78231,San Antonio,TX,210,Bexar,29.577,-98.539,234642.0,326961.0,"(300000, 400000]",3,NaN,0,Pizza Place,Pharmacy,Gym / Fitness Center,Gas Station,Video Store,Coffee Shop,Convenience Store,Spa,Cosmetics Shop,Fast Food Restaurant
48,78249,San Antonio,TX,210,Bexar,29.567,-98.606,158196.0,224169.0,"(200000, 300000]",2,12.0,0,Convenience Store,Fast Food Restaurant,Pizza Place,Sandwich Place,Ice Cream Shop,Coffee Shop,Mexican Restaurant,Sushi Restaurant,Department Store,Tex-Mex Restaurant
50,78251,San Antonio,TX,210,Bexar,29.460,-98.669,136923.0,197111.0,"(100000, 200000]",1,10.5,1,Fast Food Restaurant,Convenience Store,Mexican Restaurant,Burger Joint,Video Store,Sandwich Place,Fried Chicken Joint,Bar,Ice Cream Shop,Seafood Restaurant
52,78253,San Antonio,TX,210,Bexar,29.459,-98.747,206318.0,263034.0,"(200000, 300000]",2,10.0,4,Video Store,Real Estate Office,Park,Pharmacy,Theater,Food Service,Food Truck,Football Stadium,Food Court,Food & Drink Shop
53,78254,San Antonio,TX,210,Bexar,29.536,-98.724,163446.0,226363.0,"(200000, 300000]",2,5.5,0,Salon / Barbershop,Convenience Store,Farm,Sandwich Place,Thrift / Vintage Store,Grocery Store,Donut Shop,Pool,Pizza Place,Pharmacy


In [98]:
# create map of the clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, price, score in zip(finalFilter['Latitude'], finalFilter['Longitude'], finalFilter['Zipcode'], 
                                                finalFilter['Cluster Labels'], finalFilter['price_labels'], finalFilter['Smart Score']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) + ' Price $' + str(price) + ' Smart Score ' + str(score), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)

map_clusters

In [58]:
# Cluster 0
SA_merged.loc[SA_merged['Cluster Labels'] == 0, SA_merged.columns[[1] + list(range(5, SA_merged.shape[1]))]]

,City,Latitude,Longitude,2012-12,2019-12,price_bins,price_labels,Smart Score,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,San Antonio,29.438,-98.457,62619.0,155677.0,"(100000.0, 200000.0]",1,NaN,0,Burger Joint,Sandwich Place,Coffee Shop,Mexican Restaurant,Southern / Soul Food Restaurant,Museum,Bookstore,Deli / Bodega,Brewery,Beer Garden
8,San Antonio,29.488,-98.457,299733.0,419831.0,"(400000.0, 500000.0]",4,5.0,0,Pizza Place,Coffee Shop,Ice Cream Shop,Grocery Store,Italian Restaurant,Bakery,American Restaurant,Sporting Goods Shop,Mexican Restaurant,Frozen Yogurt Shop
10,San Antonio,29.357,-98.560,61107.0,106029.0,"(100000.0, 200000.0]",1,4.0,0,Discount Store,Sandwich Place,Mexican Restaurant,Fast Food Restaurant,Hardware Store,Fried Chicken Joint,Burger Joint,Big Box Store,Grocery Store,Gym / Fitness Center
12,San Antonio,29.512,-98.524,102969.0,164224.0,"(100000.0, 200000.0]",1,7.5,0,Mexican Restaurant,Burger Joint,Fast Food Restaurant,Pizza Place,Asian Restaurant,Thai Restaurant,Sushi Restaurant,Convenience Store,Pharmacy,Gym / Fitness Center
15,San Antonio,29.534,-98.489,156180.0,233606.0,"(200000.0, 300000.0]",2,2.0,0,Hotel,Mexican Restaurant,Department Store,American Restaurant,Clothing Store,Cosmetics Shop,Seafood Restaurant,Toy / Game Store,Sporting Goods Shop,Fast Food Restaurant
17,San Antonio,29.501,-98.398,91154.0,151694.0,"(100000.0, 200000.0]",1,4.0,0,Mexican Restaurant,Gas Station,BBQ Joint,Fried Chicken Joint,Burger Joint,Fast Food Restaurant,Taco Place,Convenience Store,Sandwich Place,Gym / Fitness Center
19,San Antonio,29.411,-98.404,65867.0,118427.0,"(100000.0, 200000.0]",1,3.5,0,Fast Food Restaurant,Convenience Store,Hotel,Discount Store,Park,Pizza Place,Seafood Restaurant,Mexican Restaurant,BBQ Joint,Fried Chicken Joint
23,San Antonio,29.312,-98.540,87983.0,139946.0,"(100000.0, 200000.0]",1,7.0,0,Convenience Store,Flea Market,Food,Burger Joint,Intersection,Sandwich Place,Shipping Store,Taco Place,Park,Mexican Restaurant
28,San Antonio,29.501,-98.574,134635.0,194267.0,"(100000.0, 200000.0]",1,3.5,0,Mexican Restaurant,Coffee Shop,Sandwich Place,Chinese Restaurant,Bar,Pharmacy,Bakery,Indian Restaurant,Pizza Place,Fried Chicken Joint
29,San Antonio,29.540,-98.552,198422.0,278380.0,"(200000.0, 300000.0]",2,6.0,0,Mexican Restaurant,Coffee Shop,Burger Joint,Sandwich Place,Chinese Restaurant,Sushi Restaurant,Bar,Grocery Store,Fast Food Restaurant,Gym


In [59]:
# Cluster 1
SA_merged.loc[SA_merged['Cluster Labels'] == 1, SA_merged.columns[[1] + list(range(5, SA_merged.shape[1]))]]

,City,Latitude,Longitude,2012-12,2019-12,price_bins,price_labels,Smart Score,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,San Antonio,29.472,-98.537,88509.0,156320.0,"(100000.0, 200000.0]",1,4.0,1,Mexican Restaurant,Discount Store,Fast Food Restaurant,Pizza Place,Convenience Store,Ice Cream Shop,Sandwich Place,Burger Joint,Coffee Shop,Gym / Fitness Center
3,San Antonio,29.397,-98.500,77524.0,137329.0,"(100000.0, 200000.0]",1,3.0,1,Mexican Restaurant,Gas Station,BBQ Joint,Seafood Restaurant,Sandwich Place,Brewery,Discount Store,Park,Beer Garden,Bar
6,San Antonio,29.422,-98.523,56376.0,90947.0,"(50000.0, 100000.0]",0,7.0,1,Mexican Restaurant,Hotel,Discount Store,Pizza Place,Gas Station,Bar,Fast Food Restaurant,Taco Place,Thrift / Vintage Store,Bakery
9,San Antonio,29.399,-98.470,77312.0,151158.0,"(100000.0, 200000.0]",1,3.0,1,Mexican Restaurant,Discount Store,Fast Food Restaurant,Restaurant,BBQ Joint,Burger Joint,Bar,Sandwich Place,Football Stadium,Snack Place
11,San Antonio,29.462,-98.494,109080.0,189900.0,"(100000.0, 200000.0]",1,8.5,1,Mexican Restaurant,Burger Joint,Bar,Park,Gay Bar,American Restaurant,Pizza Place,Coffee Shop,Taco Place,Cajun / Creole Restaurant
13,San Antonio,29.366,-98.487,66324.0,119099.0,"(100000.0, 200000.0]",1,6.5,1,Mexican Restaurant,Fast Food Restaurant,Pizza Place,Fried Chicken Joint,Burger Joint,Seafood Restaurant,Convenience Store,Sandwich Place,Chinese Restaurant,BBQ Joint
16,San Antonio,29.544,-98.420,117063.0,179911.0,"(100000.0, 200000.0]",1,3.5,1,Mexican Restaurant,Bar,Food Truck,Sandwich Place,Latin American Restaurant,Discount Store,Baseball Field,Athletics & Sports,Golf Course,Private School
18,San Antonio,29.440,-98.395,77266.0,130577.0,"(100000.0, 200000.0]",1,1.5,1,Mexican Restaurant,Hotel,Gas Station,Discount Store,Convenience Store,Sandwich Place,Pizza Place,Tourist Information Center,Fried Chicken Joint,Seafood Restaurant
20,San Antonio,29.319,-98.496,79738.0,133340.0,"(100000.0, 200000.0]",1,5.0,1,Convenience Store,Mexican Restaurant,Discount Store,Golf Course,Food,Sushi Restaurant,Moving Target,Food Truck,Sandwich Place,Ice Cream Shop
21,San Antonio,29.387,-98.392,108893.0,158224.0,"(100000.0, 200000.0]",1,2.5,1,Fast Food Restaurant,Discount Store,Convenience Store,Mexican Restaurant,Gas Station,Mobile Phone Shop,Burger Joint,BBQ Joint,Pizza Place,Construction & Landscaping


In [60]:
# Cluster 2
SA_merged.loc[SA_merged['Cluster Labels'] ==2, SA_merged.columns[[1] + list(range(5, SA_merged.shape[1]))]]

,City,Latitude,Longitude,2012-12,2019-12,price_bins,price_labels,Smart Score,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,San Antonio,29.422,-98.466,60016.0,129942.0,"(100000.0, 200000.0]",1,3.0,2,Hotel,Steakhouse,BBQ Joint,Sandwich Place,Theater,Cocktail Bar,Ice Cream Shop,Museum,Coffee Shop,Beer Garden
2,San Antonio,29.415,-98.462,71213.0,150560.0,"(100000.0, 200000.0]",1,1.5,2,Hotel,Mexican Restaurant,Burger Joint,Coffee Shop,BBQ Joint,History Museum,Hotel Bar,Steakhouse,Brewery,Sports Bar
4,San Antonio,29.424,-98.487,184158.0,259457.0,"(200000.0, 300000.0]",2,2.0,2,Hotel,Steakhouse,Bar,Theater,Sandwich Place,Plaza,Dessert Shop,Restaurant,Mexican Restaurant,Concert Hall
5,San Antonio,29.417,-98.499,NaN,NaN,NaN,NaN,NaN,2,Hotel,Mexican Restaurant,Park,Theater,Plaza,BBQ Joint,Steakhouse,Beer Garden,Cocktail Bar,Lounge
14,San Antonio,29.438,-98.481,123270.0,227805.0,"(200000.0, 300000.0]",2,NaN,2,Hotel,Bar,Burger Joint,Restaurant,Cocktail Bar,New American Restaurant,Bakery,Coffee Shop,Sandwich Place,Mexican Restaurant
72,San Antonio,29.411,-98.504,NaN,NaN,NaN,NaN,NaN,2,Hotel,Mexican Restaurant,Park,Cocktail Bar,Theater,Plaza,Beer Garden,BBQ Joint,Art Gallery,Seafood Restaurant
77,San Antonio,29.426,-98.492,NaN,NaN,NaN,NaN,NaN,2,Hotel,Mexican Restaurant,Bar,Steakhouse,Plaza,Theater,Sandwich Place,Seafood Restaurant,Cocktail Bar,Park
79,San Antonio,29.426,-98.488,NaN,NaN,NaN,NaN,NaN,2,Hotel,Steakhouse,Bar,Theater,Sandwich Place,Plaza,Cocktail Bar,Dessert Shop,Bakery,Park
80,San Antonio,29.426,-98.486,NaN,NaN,NaN,NaN,NaN,2,Hotel,Steakhouse,Bar,Cocktail Bar,Theater,Plaza,Sandwich Place,Bakery,BBQ Joint,Ice Cream Shop
81,San Antonio,29.425,-98.487,NaN,NaN,NaN,NaN,NaN,2,Hotel,Steakhouse,Bar,Theater,Cocktail Bar,Sandwich Place,Plaza,Dessert Shop,Ice Cream Shop,Park


In [61]:
# Cluster 3
SA_merged.loc[SA_merged['Cluster Labels'] == 3, SA_merged.columns[[1] + list(range(5, SA_merged.shape[1]))]]

,City,Latitude,Longitude,2012-12,2019-12,price_bins,price_labels,Smart Score,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,San Antonio,29.193,-98.517,97261.0,151077.0,"(100000, 200000]",1,NaN,3,Massage Studio,Food,Restaurant,Zoo,Food & Drink Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food Court


In [62]:
# Cluster 4
SA_merged.loc[SA_merged['Cluster Labels'] == 4, SA_merged.columns[[1] + list(range(5, SA_merged.shape[1]))]]

,City,Latitude,Longitude,2012-12,2019-12,price_bins,price_labels,Smart Score,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,San Antonio,29.407,-98.735,126281.0,186460.0,"(100000, 200000]",1,2.5,4,Video Store,Pool,Golf Course,Pharmacy,Food & Drink Shop,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant
51,San Antonio,29.349,-98.695,119458.0,177851.0,"(100000, 200000]",1,2.5,4,Residential Building (Apartment / Condo),Scenic Lookout,Lake,Pharmacy,River,Flower Shop,Fast Food Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market
52,San Antonio,29.459,-98.747,206318.0,263034.0,"(200000, 300000]",2,10.0,4,Video Store,Real Estate Office,Park,Pharmacy,Theater,Food Service,Food Truck,Football Stadium,Food Court,Food & Drink Shop
59,San Antonio,29.707,-98.483,280002.0,357993.0,"(300000, 400000]",3,NaN,4,Gas Station,Pool,American Restaurant,Home Service,Real Estate Office,Mattress Store,Zoo,Food,Fish & Chips Shop,Fish Market


In [63]:
# Cluster 5 (if applicable)
SA_merged.loc[SA_merged['Cluster Labels'] == 5, SA_merged.columns[[1] + list(range(5, SA_merged.shape[1]))]]

,City,Latitude,Longitude,2012-12,2019-12,price_bins,price_labels,Smart Score,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,San Antonio,29.354,-98.332,177750.0,252585.0,"(200000, 300000]",2,5.5,5,Construction & Landscaping,Gym,Zoo,Food & Drink Shop,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food
